# !!!!!!! CLEAR OUTPUT BEFORE COMMIT !!!!!!
TODO:
- Histogram of home age?

In [1]:
import pandas as pd
import numpy as np
import os
#import sklearn

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def one_hot_encode(df, col_name_list):
    for col_name in col_name_list:
        df = pd.concat((df, pd.get_dummies(df[col_name], prefix=col_name)), axis='columns')
    df.drop(columns=col_name_list, inplace=True)
    return df

def a_f_ohe(df, column_name, one_hot_encode_list):
    if not one_hot_encode_list:
        one_hot_encode_list = []
    df[column_name] = column_name + df[column_name].astype(str)
    if not column_name in one_hot_encode_list:
        one_hot_encode_list = one_hot_encode_list + [column_name]
    return one_hot_encode_list

In [3]:
df = pd.read_csv(os.path.join('data','train.csv'))

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


How many records are we dealing with?

In [5]:
len(df.index)

1460

Because of the small size of the training and testing datasets we don't need to worry about converting data types to smaller sizes to conserve space.

What data types are we dealing with?

In [6]:
df.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
Alley             object
LotShape          object
LandContour       object
Utilities         object
LotConfig         object
LandSlope         object
Neighborhood      object
Condition1        object
Condition2        object
BldgType          object
HouseStyle        object
OverallQual        int64
OverallCond        int64
YearBuilt          int64
YearRemodAdd       int64
RoofStyle         object
RoofMatl          object
Exterior1st       object
Exterior2nd       object
MasVnrType        object
MasVnrArea       float64
ExterQual         object
ExterCond         object
Foundation        object
BsmtQual          object
BsmtCond          object
BsmtExposure      object
BsmtFinType1      object
BsmtFinSF1         int64
BsmtFinType2      object
BsmtFinSF2         int64
BsmtUnfSF          int64
TotalBsmtSF        int64
Heating           object


Let's take a look at the different features and get some simple summary statistics

In [7]:
df['1stFlrSF'].describe()

count    1460.000000
mean     1162.626712
std       386.587738
min       334.000000
25%       882.000000
50%      1087.000000
75%      1391.250000
max      4692.000000
Name: 1stFlrSF, dtype: float64

In [8]:
df['MSSubClass'].value_counts()

20     536
60     299
50     144
120     87
30      69
160     63
70      60
80      58
90      52
190     30
85      20
75      16
45      12
180     10
40       4
Name: MSSubClass, dtype: int64

Which fields have null values?

In [9]:
df_nulls = df.copy().isnull().sum()
df_nulls = df_nulls.to_frame().rename(columns={0:'num_nulls'})
df_nulls['total_records'] = len(df.index)
df_nulls['pct_null'] = df_nulls['num_nulls'] / df_nulls['total_records'] * 100
df_nulls[df_nulls['num_nulls'] > 0].sort_values(by='pct_null', ascending=False)

,num_nulls,total_records,pct_null
PoolQC,1453,1460,99.520548
MiscFeature,1406,1460,96.301370
Alley,1369,1460,93.767123
Fence,1179,1460,80.753425
FireplaceQu,690,1460,47.260274
LotFrontage,259,1460,17.739726
GarageType,81,1460,5.547945
GarageYrBlt,81,1460,5.547945
GarageFinish,81,1460,5.547945
GarageQual,81,1460,5.547945


Wow, the PoolQC (Pool Quality) is almost always empty. Is this because:
- Most of the houses have no pools?
- How many houses have pools but no quality value?

In [10]:
df.loc[df['PoolArea'] > 0, ['PoolQC','PoolArea']]

,PoolQC,PoolArea
197,Ex,512
810,Fa,648
1170,Gd,576
1182,Ex,555
1298,Gd,480
1386,Fa,519
1423,Gd,738


There are no pools without a quality rating.

In [11]:
df.loc[~df['PoolQC'].isnull(), ['PoolQC','PoolArea']]

,PoolQC,PoolArea
197,Ex,512
810,Fa,648
1170,Gd,576
1182,Ex,555
1298,Gd,480
1386,Fa,519
1423,Gd,738


This shows us that only 7 houses had a pool and no pool quality records exist where there is no pool (pool area = 0)

Pools normally represent a small amount of houses so it's important to keep this around rather than attempting to just drop it since there are so few records.

To handle these we can One Hot Encode (pd.get_dummies) for the pool quality values. Any house without a pool will have 0 for all the quality measurements to work around the null values in the pool quality feature.

If we're doing something similar to the PoolQC we should explicitly show it like we did with PoolQC

In [12]:
ohe_list = [] #['PoolQC','Alley','Fence','MiscFeature']

In [13]:
#df = one_hot_encode(df, ohe_list)

In [14]:
#df.head()

**MSSubClass:** Identifies the type of dwelling involved in the sale.

In [15]:
# Prefix the categories so they make a bit more sense when we OHE them.
# Initially it's imported as a number so we'll force it to be a string
ohe_list = a_f_ohe(df, 'MSSubClass', ohe_list)

**MSZoning:** Identifies the general zoning classification of the sale.

In [16]:
ohe_list = a_f_ohe(df, 'MSZoning', ohe_list)

**LotFrontage:** Linear feet of street connected to property

Nothing to do here. This is a simple number. Perhaps scale?

**LotFrontage:** Linear feet of street connected to property

Same as above

**Street:** Type of road access to property

In [17]:
ohe_list = a_f_ohe(df, 'Street', ohe_list)

**Alley:** Type of alley access to property

In [18]:
ohe_list = a_f_ohe(df, 'Alley', ohe_list)

**LotShape:** General shape of property

In [19]:
ohe_list = a_f_ohe(df, 'LotShape', ohe_list)

**LandContour:** Flatness of the property

In [20]:
ohe_list = a_f_ohe(df, 'LandContour', ohe_list)

**Utilities:** Type of utilities available

In [21]:
ohe_list = a_f_ohe(df, 'Utilities', ohe_list)

**LotConfig:** Lot configuration

In [22]:
ohe_list = a_f_ohe(df, 'LotConfig', ohe_list)

**LandSlope:** Slope of property

In [23]:
ohe_list = a_f_ohe(df, 'LandSlope', ohe_list)

**Neighborhood:** Physical locations within Ames city limits

In [24]:
ohe_list = a_f_ohe(df, 'Neighborhood', ohe_list)

**Condition1:** Proximity to various conditions

Are these mutually exclusive values? They appear so.

In [25]:
df['Condition1'].value_counts()

Norm      1260
Feedr       81
Artery      48
RRAn        26
PosN        19
RRAe        11
PosA         8
RRNn         5
RRNe         2
Name: Condition1, dtype: int64

In [26]:
ohe_list = a_f_ohe(df, 'Condition1', ohe_list)

**Condition2:** Proximity to various conditions (if more than one is present)

In [27]:
df['Condition2'].value_counts()

Norm      1445
Feedr        6
RRNn         2
Artery       2
PosN         2
PosA         1
RRAn         1
RRAe         1
Name: Condition2, dtype: int64

In [28]:
ohe_list = a_f_ohe(df, 'Condition2', ohe_list)

**BldgType:** Type of dwelling

In [29]:
ohe_list = a_f_ohe(df, 'BldgType', ohe_list)

**HouseStyle:** Style of dwelling

In [30]:
ohe_list = a_f_ohe(df, 'HouseStyle', ohe_list)

**OverallQual:** Rates the overall material and finish of the house

Rating between 1 and 10 (Very Poor and Very Excellent) - Since this is numeric and has a natural relationship we'll leave alone.

**OverallCond:** Rates the overall condition of the house

Rating between 1 and 10 (Very Poor and Very Excellent) - Since this is numeric and has a natural relationship we'll leave alone.

**YearBuilt:** Original construction date

Convert this to be the ages in full years

In [31]:
from datetime import datetime
df['Age'] = datetime.now().year - df['YearBuilt']
del df['YearBuilt']

**YearRemodAdd:** Remodel date (same as construction date if no remodeling or additions)

This is the same process as the YearBuilt

In [32]:
df['RemodelAge'] = datetime.now().year - df['YearRemodAdd']
del df['YearRemodAdd']

**RoofStyle:** Type of roof

In [33]:
ohe_list = a_f_ohe(df, 'RoofStyle', ohe_list)

**RoofMatl:** Roof material

In [34]:
ohe_list = a_f_ohe(df, 'RoofMatl', ohe_list)

**Exterior1st:** Exterior covering on house

In [35]:
ohe_list = a_f_ohe(df, 'Exterior1st', ohe_list)

**Exterior2nd:** Exterior covering on house (if more than one material)

In [36]:
ohe_list = a_f_ohe(df, 'Exterior2nd', ohe_list)

**MasVnrType:** Masonry veneer type

In [37]:
ohe_list = a_f_ohe(df, 'MasVnrType', ohe_list)

**MasVnrArea:** Masonry veneer area in square feet

Nothing to do here, simple number. Perhaps scale?

**ExterQual:** Evaluates the quality of the material on the exterior

In [38]:
ohe_list = a_f_ohe(df, 'ExterQual', ohe_list)

**ExterQual:** Evaluates the quality of the material on the exterior

In [39]:
ohe_list = a_f_ohe(df, 'ExterQual', ohe_list)

**ExterCond:** Evaluates the present condition of the material on the exterior

In [40]:
ohe_list = a_f_ohe(df, 'ExterCond', ohe_list)

**Foundation:** Type of foundation

In [41]:
ohe_list = a_f_ohe(df, 'Foundation', ohe_list)

**BsmtQual:** Evaluates the height of the basement

In [42]:
ohe_list = a_f_ohe(df, 'BsmtQual', ohe_list)

**BsmtCond:** Evaluates the general condition of the basement

In [43]:
ohe_list = a_f_ohe(df, 'BsmtCond', ohe_list)

**BsmtExposure:** Refers to walkout or garden level walls

In [44]:
ohe_list = a_f_ohe(df, 'BsmtExposure', ohe_list)

**BsmtFinType1:** Rating of basement finished area

In [45]:
ohe_list = a_f_ohe(df, 'BsmtFinType1', ohe_list)

**BsmtFinSF1:** Type 1 finished square feet
Numeric field. Perhaps Scale?

**BsmtFinType2:** Rating of basement finished area (if multiple types)

In [46]:
ohe_list = a_f_ohe(df, 'BsmtFinType2', ohe_list)

**BsmtFinSF2:** Type 2 finished square feet

Number - Scale?

**BsmtUnfSF:** Unfinished square feet of basement area

Number - scale?

**TotalBsmtSF:** Total square feet of basement area

Number - Scale?

**Heating:** Type of heating

In [47]:
ohe_list = a_f_ohe(df, 'Heating', ohe_list)

**HeatingQC:** Heating quality and condition

In [48]:
ohe_list = a_f_ohe(df, 'HeatingQC', ohe_list)

**CentralAir:** Central air conditioning

Boolean. Convery Y/N to 1/0

In [49]:
df['CentralAir'] = df['CentralAir'].map({'Y':1,'N':0}).astype(int)

**Electrical:** Electrical system

In [50]:
ohe_list = a_f_ohe(df, 'Electrical', ohe_list)

**1stFlrSF:** First Floor square feet

**2ndFlrSF:** Second floor square feet
 
**LowQualFinSF:** Low quality finished square feet (all floors)
 
**GrLivArea:** Above grade (ground) living area square feet
 
**BsmtFullBath:** Basement full bathrooms
 
**BsmtHalfBath:** Basement half bathrooms
 
**FullBath:** Full bathrooms above grade
 
**HalfBath:** Half baths above grade
 
**Bedroom:** Bedrooms above grade (does NOT include basement bedrooms)
 
**Kitchen:** Kitchens above grade

All numerical - not much to do here. Probably scale?

**KitchenQual:** Kitchen quality

In [53]:
ohe_list = a_f_ohe(df, 'KitchenQual', ohe_list)

**TotRmsAbvGrd:** Total rooms above grade (does not include bathrooms)

Number - not much to do here. Scale?

**Functional:** Home functionality (Assume typical unless deductions are warranted)

In [54]:
ohe_list = a_f_ohe(df, 'Functional', ohe_list)

**Fireplaces:** Number of fireplaces

Number, not much to do. Scale?

**FireplaceQu:** Fireplace quality

In [55]:
ohe_list = a_f_ohe(df, 'FireplaceQu', ohe_list)

**GarageType:** Garage location

In [52]:
ohe_list = a_f_ohe(df, 'GarageType', ohe_list)

**GarageYrBlt**: Year garage was built

Year - Convert to Age in years.

In [56]:
df['GarageAge'] = datetime.now().year - df['GarageYrBlt']
del df['GarageYrBlt']

**GarageFinish:** Interior finish of the garage

In [57]:
ohe_list = a_f_ohe(df, 'GarageFinish', ohe_list)

**GarageCars:** Size of garage in car capacity
 
**GarageArea:** Size of garage in square feet

Numeric fields - scale?

**GarageQual:** Garage quality

In [60]:
ohe_list = a_f_ohe(df, 'GarageQual', ohe_list)

**GarageCond:** Garage condition

In [58]:
ohe_list = a_f_ohe(df, 'GarageCond', ohe_list)

**PavedDrive:** Paved driveway

In [59]:
ohe_list = a_f_ohe(df, 'PavedDrive', ohe_list)

**WoodDeckSF:** Wood deck area in square feet
 
**OpenPorchSF:** Open porch area in square feet
 
**EnclosedPorch:** Enclosed porch area in square feet
 
**3SsnPorch:** Three season porch area in square feet
 
**ScreenPorch:** Screen porch area in square feet
 
**PoolArea:** Pool area in square feet

Numeric, nothing to do.

**PoolQC:** Pool quality

In [62]:
ohe_list = a_f_ohe(df, 'PoolQC', ohe_list)

**Fence:** Fence quality

In [61]:
ohe_list = a_f_ohe(df, 'Fence', ohe_list)

**MiscFeature:** Miscellaneous feature not covered in other categories

In [63]:
ohe_list = a_f_ohe(df, 'MiscFeature', ohe_list)

**MiscVal:** $Value of miscellaneous feature

Numeric - Nothing to do

**MoSold:** Month Sold (MM)

Do we need to do anything with the month?

**YrSold:** Year Sold (YYYY)

Should we do anything with the year sold? Convert it to an age? Combine it with month to get fractional years? Then drop month and year? Keep month?

**SaleType:** Type of sale

In [64]:
ohe_list = a_f_ohe(df, 'SaleType', ohe_list)

**SaleCondition:** Condition of sale

In [65]:
ohe_list = a_f_ohe(df, 'SaleCondition', ohe_list)